In [ ]:
#default_exp esol
#all_slow

# esol
> Using molmapnets for regression, with descriptors, or fingerprints, or both. Tested on the [eSOL](http://www.tanpaku.org/tp-esol/index.php?lang=en) dataset.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_theme(palette='Set2')
colors = sns.color_palette()
colors

[(0.4, 0.7607843137254902, 0.6470588235294118),
 (0.9882352941176471, 0.5529411764705883, 0.3843137254901961),
 (0.5529411764705883, 0.6274509803921569, 0.796078431372549),
 (0.9058823529411765, 0.5411764705882353, 0.7647058823529411),
 (0.6509803921568628, 0.8470588235294118, 0.32941176470588235),
 (1.0, 0.8509803921568627, 0.1843137254901961),
 (0.8980392156862745, 0.7686274509803922, 0.5803921568627451),
 (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]

In [ ]:
#export
import torch
from torch import nn, optim
import torch.nn.functional as F
torch.set_default_dtype(torch.float64)

from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
from chembench import dataset
from molmap import MolMap

In [ ]:
from molmapnets.data import SingleFeatureData, DoubleFeatureData
from molmapnets.models import MolMapRegression

RDKit WARNING: [21:38:06] Enabling RDKit 2019.09.3 jupyter extensions


## Feature extraction 

The `chembench` package collected several different datasets for benchmarking the models. Here we'll use the [`eSOL`](http://www.tanpaku.org/tp-esol/index.php?lang=en) dataset, which collects the solubility of all E.coli proteins. The data can be loaded with

In [ ]:
data = dataset.load_ESOL()

total samples: 1128


We have the smiles (Simplified Molecular Input Line Entry Specification) for different proteins and their corresponding solubility meansure:

In [ ]:
data.df.head()

,smiles,measured log solubility in mols per litre
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.77
1,Cc1occc1C(=O)Nc2ccccc2,-3.30
2,CC(C)=CCCC(C)=CC(=O),-2.06
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.87
4,c1ccsc1,-1.33


Using MolMap we can extract features from using the smiles as input. We can specify the feature type `ftype`,  feature pairwise distance calculation method `metric`, and feature grid arrangement method `fmap_type`:

In [ ]:
MolMap?

Init signature:
MolMap(
    ftype='descriptor',
    flist=None,
    fmap_type='grid',
    fmap_shape=None,
    split_channels=True,
    metric='cosine',
    var_thr=0.0001,
)
Docstring:      <no docstring>
Init docstring:
paramters
-----------------
ftype: {'fingerprint', 'descriptor'}, feature type
flist: feature list, if you want use some of the features instead of all features, each element in flist should be the id of a feature
fmap_shape: None or tuple, size of molmap, only works when fmap_type is 'scatter', if None, the size of feature map will be calculated automatically
fmap_type:{'scatter', 'grid'}, default: 'gird', if 'scatter', will return a scatter mol map without an assignment to a grid
split_channels: bool, if True, outputs will split into various channels using the types of feature
metric: {'cosine', 'correlation'}, default: 'cosine', measurement of feature distance
var_thr: float, defalt is 1e-4, meaning that feature will be included only if the conresponding variance l

In [ ]:
descriptor = MolMap(ftype='descriptor', metric='cosine',)

In [ ]:
fingerprint = MolMap(ftype='fingerprint', metric='cosine')

After setting up the feature extracting method, we can then use the `.fit` method of the feature object to extract the features. During this step we need to specify the algorithm (`method`) to embed higher dimensional features to 2D presentation: 

In [ ]:
descriptor.fit(verbose=0, method='umap', min_dist=0.1, n_neighbors=15,)

2021-07-22 21:38:23,335 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2021-07-22 21:38:26,510 - INFO - [bidd-molmap] - Finished


In [ ]:
fingerprint.fit(verbose=0, method='umap', min_dist=0.1, n_neighbors=10,)

/Users/olivier/.local/lib/python3.6/site-packages/umap/umap_.py:1461: UserWarning: Using precomputed metric; transform will be unavailable for new data
  "Using precomputed metric; transform will be unavailable for new data"


2021-07-22 21:38:54,356 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2021-07-22 22:11:46,397 - INFO - [bidd-molmap] - Finished


We can visualise the feature maps easily with MolMap, but the visualisations are removed to avoid crushing the notebook.

## Regression using the descriptor map

In [ ]:
X = descriptor.batch_transform(data.x)

100%|##########| 1128/1128 [08:45<00:00,  2.35it/s]


In [ ]:
X.shape

(1128, 37, 37, 13)

In PyTorch the training data for computer vision problems takes the shape `(n_channels, hight, width)`, while the features extracted from `MolMap` take the shape `(hight, width, n_channels)`, so we'll first correct it by moving the channels dimension before the feature map dimensions.

In [ ]:
torch.movedim(torch.from_numpy(X), -1, 1).shape

torch.Size([1128, 13, 37, 37])

In [ ]:
Y = data.y

In [ ]:
Y.shape

(1128, 1)

Now from these feature maps we can create the dataset suitable for training models in PyTorch

In [ ]:
esol = SingleFeatureData(data.y, X)

In [ ]:
train, val, test = random_split(esol, [904,112,112], generator=torch.Generator().manual_seed(7))

In [ ]:
len(train), len(val), len(test)

(904, 112, 112)

In [ ]:
train_loader = DataLoader(train, batch_size=8, shuffle=True)
val_loader = DataLoader(val, batch_size=8, shuffle=True)
test_loader = DataLoader(test, batch_size=8, shuffle=True)

And we can get one batch of data by making the data loader iterable

In [ ]:
x, t = next(iter(train_loader))

In [ ]:
t

tensor([[-2.3560],
        [-2.4840],
        [-1.0600],
        [ 0.3900],
        [ 0.9600],
        [-2.1700],
        [-2.7300],
        [-0.6200]])

In [ ]:
x.shape

torch.Size([8, 13, 37, 37])

Finally with the data prepared we can train the models. These are tests to show that the models work as expected, but we can certainly fine tune the model to achieve better results.

In [ ]:
model = MolMapRegression()

epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

And the training loop

In [ ]:
for epoch in range(epochs):

    running_loss = 0.0
    for i, (xb, yb) in enumerate(train_loader):

        xb, yb = xb.to(device), yb.to(device)

        # zero gradients
        optimizer.zero_grad()

        # forward propagation
        pred = model(xb)

        # loss calculation
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 50 == 0:    
            print('[Epoch: %d, Iter: %5d] Training loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / (i+1)))

print('Training finished')

/Users/olivier/opt/anaconda3/envs/molmap/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch: 1, Iter:    50] Training loss: 6.567
[Epoch: 1, Iter:   100] Training loss: 5.444
[Epoch: 2, Iter:    50] Training loss: 3.304
[Epoch: 2, Iter:   100] Training loss: 2.579
[Epoch: 3, Iter:    50] Training loss: 2.061
[Epoch: 3, Iter:   100] Training loss: 1.807
[Epoch: 4, Iter:    50] Training loss: 1.179
[Epoch: 4, Iter:   100] Training loss: 1.294
[Epoch: 5, Iter:    50] Training loss: 1.191
[Epoch: 5, Iter:   100] Training loss: 1.397
Training finished


Loss on validation data set

In [ ]:
running_loss = 0.0
with torch.no_grad():
    for i, (xb, yb) in enumerate(val_loader):

        xb, yb = xb.to(device), yb.to(device)

        # forward propagation
        pred = model(xb)

        # loss calculation
        loss = criterion(pred, yb)
        running_loss += loss.item()
        if (i+1) % 3 == 0:    
            print('[Iter: %5d] Validation loss: %.3f' %
                    (i + 1, running_loss / (i+1)))


[Iter:     3] Validation loss: 0.732
[Iter:     6] Validation loss: 0.950
[Iter:     9] Validation loss: 0.959
[Iter:    12] Validation loss: 0.983


## Regression using the fingerprint map

In [ ]:
X_fingerprint = fingerprint.batch_transform(data.x)

100%|##########| 1128/1128 [03:37<00:00,  5.55it/s]


In [ ]:
X_fingerprint.shape

(1128, 126, 126, 12)

Now from these feature maps we can create the dataset suitable for training models in PyTorch

In [ ]:
esol_fingerprint = SingleFeatureData(data.y, X_fingerprint)

In [ ]:
train_fingerprint, val_fingerprint, test_fingerprint = random_split(esol_fingerprint, [904,112,112], generator=torch.Generator().manual_seed(7))

In [ ]:
len(train), len(val), len(test)

(904, 112, 112)

In [ ]:
train_loader_fingerprint = DataLoader(train_fingerprint, batch_size=8, shuffle=True)
val_loader_fingerprint = DataLoader(val_fingerprint, batch_size=8, shuffle=True)
test_loader_fingerprint = DataLoader(test_fingerprint, batch_size=8, shuffle=True)

And we can get one batch of data by making the data loader iterable

In [ ]:
x, t = next(iter(train_loader_fingerprint))

In [ ]:
t.shape

torch.Size([8, 1])

In [ ]:
x.shape

torch.Size([8, 12, 126, 126])

And regression. Different feature maps have different number of channels.

In [ ]:
model_fingerprint = MolMapRegression(conv_in1=12)

epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_fingerprint.to(device)
optimizer = optim.Adam(model_fingerprint.parameters(), lr=0.001)
criterion = nn.MSELoss()

And the training loop

In [ ]:
for epoch in range(epochs):

    running_loss = 0.0
    for i, (xb, yb) in enumerate(train_loader_fingerprint):

        xb, yb = xb.to(device), yb.to(device)

        # zero gradients
        optimizer.zero_grad()

        # forward propagation
        pred = model_fingerprint(xb)

        # loss calculation
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 50 == 0:    
            print('[Epoch: %d, Iter: %5d] Training loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / (i+1)))

print('Training finished')

[Epoch: 1, Iter:    50] Training loss: 4.537
[Epoch: 1, Iter:   100] Training loss: 3.935
[Epoch: 2, Iter:    50] Training loss: 2.271
[Epoch: 2, Iter:   100] Training loss: 1.954
[Epoch: 3, Iter:    50] Training loss: 1.258
[Epoch: 3, Iter:   100] Training loss: 1.157
[Epoch: 4, Iter:    50] Training loss: 1.027
[Epoch: 4, Iter:   100] Training loss: 0.880
[Epoch: 5, Iter:    50] Training loss: 0.660
[Epoch: 5, Iter:   100] Training loss: 0.649
Training finished


Loss on validation data set

In [ ]:
running_loss = 0.0
with torch.no_grad():
    for i, (xb, yb) in enumerate(val_loader_fingerprint):

        xb, yb = xb.to(device), yb.to(device)

        # forward propagation
        pred = model_fingerprint(xb)

        # loss calculation
        loss = criterion(pred, yb)
        running_loss += loss.item()
        if (i+1) % 3 == 0:    
            print('[Iter: %5d] Validation loss: %.3f' %
                    (i + 1, running_loss / (i+1)))


[Iter:     3] Validation loss: 0.442
[Iter:     6] Validation loss: 0.599
[Iter:     9] Validation loss: 0.708
[Iter:    12] Validation loss: 0.851


## Regression using both feature maps

If we want to use both the feature maps, we have to process the training data differently.

Now we can feed both the feature maps to the model as a tuple

In [ ]:
double_feature = DoubleFeatureData(data.y, (X, X_fingerprint))

In [ ]:
train_double, val_double, test_double = random_split(double_feature, [904,112,112], generator=torch.Generator().manual_seed(7))

In [ ]:
len(train_double), len(val_double), len(test_double)

(904, 112, 112)

In [ ]:
train_loader_double = DataLoader(train_double, batch_size=8, shuffle=True)
val_loader_double = DataLoader(val_double, batch_size=8, shuffle=True)
test_loader_double = DataLoader(test_double, batch_size=8, shuffle=True)

And we can get one batch of data by making the data loader iterable

In [ ]:
x, t = next(iter(train_loader_double))

In [ ]:
t.shape

torch.Size([8, 1])

In [ ]:
x1, x2 = x
x1.shape, x2.shape

(torch.Size([8, 13, 37, 37]), torch.Size([8, 12, 126, 126]))

And regression. Different feature maps have different number of channels.

In [ ]:
model_double = MolMapRegression(conv_in1=13, conv_in2=12)

epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_double.to(device)
optimizer = optim.Adam(model_double.parameters(), lr=0.001)
criterion = nn.MSELoss()

And the training loop

In [ ]:
for epoch in range(epochs):

    running_loss = 0.0
    for i, ((x1, x2), yb) in enumerate(train_loader_double):

        x1, x2, yb = x1.to(device), x2.to(device), yb.to(device)

        # zero gradients
        optimizer.zero_grad()

        # forward propagation
        pred = model_double((x1, x2))

        # loss calculation
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i+1) % 50 == 0:    
            print('[Epoch: %d, Iter: %5d] Training loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / (i+1)))

print('Training finished')

[Epoch: 1, Iter:    50] Training loss: 6.050
[Epoch: 1, Iter:   100] Training loss: 4.608
[Epoch: 2, Iter:    50] Training loss: 2.854
[Epoch: 2, Iter:   100] Training loss: 2.403
[Epoch: 3, Iter:    50] Training loss: 1.834
[Epoch: 3, Iter:   100] Training loss: 1.536
[Epoch: 4, Iter:    50] Training loss: 1.027
[Epoch: 4, Iter:   100] Training loss: 0.996
[Epoch: 5, Iter:    50] Training loss: 0.595
[Epoch: 5, Iter:   100] Training loss: 0.677
Training finished


Loss on validation data set

In [ ]:
running_loss = 0.0
with torch.no_grad():
    for i, ((x1, x2), yb) in enumerate(val_loader_double):

        x1, x2, yb = x1.to(device), x2.to(device), yb.to(device)

        # forward propagation
        pred = model_double((x1, x2))

        # loss calculation
        loss = criterion(pred, yb)
        running_loss += loss.item()
        if (i+1) % 3 == 0:    
            print('[Iter: %5d] Validation loss: %.3f' %
                    (i + 1, running_loss / (i+1)))

print('Validation finished')

[Iter:     3] Validation loss: 1.191
[Iter:     6] Validation loss: 1.088
[Iter:     9] Validation loss: 1.002
[Iter:    12] Validation loss: 0.874
Validation finished
